In [1]:
import os
os.environ['JAVA_HOME'] = '/opt/homebrew/Cellar/openjdk/22.0.2/libexec/openjdk.jdk/Contents/Home'

## Importing libraries

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

In [2]:
# Initialize spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 1 raw data transformation")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "8g") 
    .config("spark.executor.memory", "8g")
    .getOrCreate()
)

24/08/16 17:04:06 WARN Utils: Your hostname, Phams-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.16.47.168 instead (on interface en0)
24/08/16 17:04:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/16 17:04:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Merging monthly parquet files into 1 file

In [3]:
# Merge all monthly records into one dataframe
sdf_yellow = spark.read.parquet('../data/landing/2023-01.parquet')

for month in range(2, 13):
    month = str(month).zfill(2)
    df_next = spark.read.parquet(f'../data/landing/2023-{month}.parquet')
    sdf_yellow = sdf_yellow.union(df_next)
    
original_count = sdf_yellow.count()

## Filtering entries

Checking data description

In [4]:
# Add new columns from existing columns
sdf_yellow = sdf_yellow.withColumn('trip_duration_minutes',
                   (F.unix_timestamp('tpep_dropoff_datetime').cast('long') - 
                    F.unix_timestamp('tpep_pickup_datetime').cast('long')) / 60)

sdf_yellow = sdf_yellow.withColumn('pickup_date', F.to_date(sdf_yellow['tpep_pickup_datetime']))
sdf_yellow = sdf_yellow.withColumn('pickup_hour', F.hour(sdf_yellow['tpep_pickup_datetime']))

sdf_yellow = sdf_yellow.withColumn('dropoff_date', F.to_date(sdf_yellow['tpep_dropoff_datetime']))

# Check earliest and latest of datetime variables
print("tpep_pickup_datetime:\n\tMax:", sdf_yellow.agg({"tpep_pickup_datetime": "max"}).collect()[0][0],
     "\n\tMin:", sdf_yellow.agg({"tpep_pickup_datetime": "min"}).collect()[0][0])

print("\ntpep_dropoff_datetime:\n\tMax:", sdf_yellow.agg({"tpep_dropoff_datetime": "max"}).collect()[0][0],
     "\n\tMin:", sdf_yellow.agg({"tpep_dropoff_datetime": "min"}).collect()[0][0], '\n')

# Check other variables
sdf_yellow.select(['passenger_count',
           'trip_distance',
           'trip_duration_minutes',
           'fare_amount',
           'extra',
           'mta_tax',
           'tip_amount',
           'tolls_amount',
           'improvement_surcharge',
           'total_amount',
           'congestion_surcharge']).describe().show(vertical=True)

tpep_pickup_datetime:
	Max: 2024-01-03 19:42:57 
	Min: 2001-01-01 00:06:49

tpep_dropoff_datetime:
	Max: 2024-01-03 20:15:55 
	Min: 1970-01-20 10:16:32 



24/08/16 17:04:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------------------
 summary               | count                
 passenger_count       | 37000870             
 trip_distance         | 38310226             
 trip_duration_minutes | 38310226             
 fare_amount           | 38310226             
 extra                 | 38310226             
 mta_tax               | 38310226             
 tip_amount            | 38310226             
 tolls_amount          | 38310226             
 improvement_surcharge | 38310226             
 total_amount          | 38310226             
 congestion_surcharge  | 37000870             
-RECORD 1-------------------------------------
 summary               | mean                 
 passenger_count       | 1.3704258575541601   
 trip_distance         | 4.088946216083186    
 trip_duration_minutes | 16.716849137965173   
 fare_amount           | 19.522250636423735   
 extra                 | 1.5560566439884764   
 mta_tax               | 0.4856166729478451   
 tip_amount  

Filtering for invalid trips according to preprocessing specifications

In [5]:
valid_trips_only_yellow = sdf_yellow.filter(F.col('passenger_count') > 0) \
                                    .filter(F.col('trip_distance') >= 0.5) \
                                    .filter(F.col('trip_duration_minutes') >= 1) \
                                    .filter(F.col('fare_amount') >= 2.5) \
                                    .filter(F.col('extra') >= 0) \
                                    .filter(F.col('mta_tax') >= 0) \
                                    .filter(F.col('tip_amount') >= 0) \
                                    .filter(F.col('tolls_amount') >= 0) \
                                    .filter(F.col('improvement_surcharge') >= 0) \
                                    .filter(F.col('total_amount') >= 2.5) \
                                    .filter((F.col('pickup_date') >= '2023-01-01') & (F.col('pickup_date') <= '2023-12-31')) \
                                    .filter((F.col('dropoff_date') >= '2023-01-01') & (F.col('dropoff_date') <= '2023-12-31'))

Handling outliers and capping trip duration at 5 hours (300 minutes)

In [6]:
valid_trips_only_yellow = valid_trips_only_yellow.where((F.col('fare_amount') <= valid_trips_only_yellow.selectExpr('percentile(fare_amount, 0.9999)').collect()[0][0]) &
                (F.col('trip_distance') <= valid_trips_only_yellow.selectExpr('percentile(trip_distance, 0.9999)').collect()[0][0]) &
                (F.col('tip_amount') <= valid_trips_only_yellow.selectExpr('percentile(tip_amount, 0.9999)').collect()[0][0]) &
                (F.col('total_amount') <= valid_trips_only_yellow.selectExpr('percentile(total_amount, 0.9999)').collect()[0][0]) &
                (F.col('tolls_amount') <= valid_trips_only_yellow.selectExpr('percentile(tolls_amount, 0.9999)').collect()[0][0]) &
                (F.col('trip_duration_minutes') <= 300))

24/08/16 17:04:23 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


After filtering for valid trips only and handling outliers, approximately 11% of the original records were removed

In [7]:
processed_count = valid_trips_only_yellow.count()
print("Number of entries deleted: " + str(original_count - processed_count))
print("Proportion of data kept: " + str(processed_count/original_count))

Number of entries deleted: 4047740
Proportion of data kept: 0.89434309262493


Writing data into `data/raw` directory

In [8]:
output_dir = '../data/raw/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [9]:
valid_trips_only_yellow.write.mode('overwrite').parquet(f'{output_dir}')